In [38]:
#library imports
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium

In [23]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text  #to import the data
soup = BeautifulSoup(url,'lxml')


In [24]:
#to store the data
postcode = []
borough = []
neighbourhood = []

In [25]:
table_find = soup.find('table')
#for finding table data
table_data = table_find.find_all('td')


#code to store the table data into dataframe
for i in range(0, len(table_data),3):
    postcode.append(table_data[i].text.strip())
    borough.append(table_data[i+1].text.strip())
    neighbourhood.append(table_data[i+2].text.strip())
df=pd.DataFrame(data=[postcode,borough,neighbourhood]).transpose()
df.columns=['postcode','borough','neighbourhood']
df.head()


,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Removal of not assigned

In [26]:
df['borough'].replace('Not assigned',np.nan,inplace=True)
df.dropna(subset=['borough'],inplace=True)
df.head()

,postcode,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# Join the Two neighbours with same postcode

In [27]:
df1=df.groupby(['postcode','borough'])['neighbourhood'].apply(','.join).reset_index()
df1.columns=['postcode','borough','neighbourhood']
df1.head()


,postcode,borough,neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [28]:
df1.shape

(103, 3)

In [29]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.columns = ['postcode', 'Latitude', 'Longitude']

In [32]:
df_pos=pd.merge(df1, df2, on=['postcode'], how='inner')

df_tor = df_pos[['borough', 'neighbourhood', 'postcode', 'Latitude', 'Longitude']].copy()

df_tor.head()

,borough,neighbourhood,postcode,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [33]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))


C:\Users\seema\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['borough'], df_tor['neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto